## Tuning di merging lists
Usa classe del prof per fsare recommender, è di hybridV4


In [5]:
import sys
sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from myRecommenders.UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from myRecommenders.ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from WeightedHybridV10 import WeightedHybridScoreRecommender
from FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg

%matplotlib inline 

import numpy as np

In [6]:
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

In [7]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout


## CHECK performance
### V3V4 sarà ibrido con 6 recommenders
### V3 è ibrido con 9 recommenders, è obsoleto

In [8]:
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

URM_train2, URM_test2 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test2 = EvaluatorHoldout(URM_test2, cutoff_list=[10])

URM_train3, URM_test3 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test3 = EvaluatorHoldout(URM_test3, cutoff_list=[10])

In [10]:
#devo rifittare con nuovo train/test split
userknn = {}
userknn["topK"] = 121
userknn["shrink"] = 2

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565


itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

cfw_icm =  {}
cfw_icm["topK"] = 702
cfw_icm["add_zeros_quota"] = 0.975154924862614
cfw_icm['normalize_similarity'] = False



recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender,
    CFW_D_Similarity_Linalg,
    ]

fits = [alpha,beta,userknn,cy,iicm,ials,itemcbf,cfw_icm]



In [11]:

only_URM = {
        "URM_train" : URM_train
    }

cython = {
        "URM_train" : URM_train,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train,
        "ICM_train" : ICM_all
    }

im = ItemIcmKNNCFRecommender(URM_train, ICM_all)
im.fit(**iicm)

cfw  =  {
        "URM_train" : URM_train,
        "ICM" : ICM_all,
        "S_matrix_target": im.W_sparse
    }

inits = [
        only_URM,only_URM,only_URM,cython,also_ICM,only_URM,CBF,cfw
    ]

ItemKNNCFRecommender: URM Detected 69 (0.87 %) cold users.
ItemKNNCFRecommender: URM Detected 2415 (9.30 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2230.67 column/sec, elapsed time 0.19 min
Similarity column 25975 ( 100 % ), 1229.42 column/sec, elapsed time 0.35 min


In [13]:

only_URM2 = {
        "URM_train" : URM_train2
    }

cython2 = {
        "URM_train" : URM_train2,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM2 = {
        "URM_train" : URM_train2,
        "ICM" : ICM_all
    }

CBF2 = {
        "URM_train" : URM_train2,
        "ICM_train" : ICM_all
    }
im2 = ItemIcmKNNCFRecommender(URM_train2, ICM_all)
im2.fit(**iicm)

cfw2  =  {
        "URM_train" : URM_train2,
        "ICM" : ICM_all,
        "S_matrix_target": im2.W_sparse
    }
inits2 = [
        only_URM2,only_URM2,only_URM2,cython2,also_ICM2,only_URM2,CBF2,cfw2
    ]

ItemKNNCFRecommender: URM Detected 72 (0.91 %) cold users.
ItemKNNCFRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2286.31 column/sec, elapsed time 0.19 min
Similarity column 25975 ( 100 % ), 1190.03 column/sec, elapsed time 0.36 min


In [15]:

only_URM3 = {
        "URM_train" : URM_train3
    }

cython3 = {
        "URM_train" : URM_train3,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM3 = {
        "URM_train" : URM_train3,
        "ICM" : ICM_all
    }

CBF3 = {
        "URM_train" : URM_train3,
        "ICM_train" : ICM_all
    }
im3 = ItemIcmKNNCFRecommender(URM_train3, ICM_all)
im3.fit(**iicm)

cfw3  =  {
        "URM_train" : URM_train3,
        "ICM" : ICM_all,
        "S_matrix_target": im3.W_sparse
    }
inits3 = [
        only_URM2,only_URM2,only_URM2,cython2,also_ICM2,only_URM2,CBF2,cfw2
    ]

ItemKNNCFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCFRecommender: URM Detected 2412 (9.29 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2325.58 column/sec, elapsed time 0.19 min
Similarity column 25975 ( 100 % ), 976.20 column/sec, elapsed time 0.44 min


In [17]:
# recsToFitV3V4_1 = []
# for rec, init in zip(recsv3v4, initsV3V4_1):
#     recsToFitV3V4_1.append(rec(**init))
# for rec, fit in zip(recsToFitV3V4_1, fitsv3v4):
#     rec.fit(**fit)
    
# recsToFitV3V4_2 = []
# for rec, init in zip(recsv3v4, initsV3V4_2):
#     recsToFitV3V4_2.append(rec(**init))
# for rec, fit in zip(recsToFitV3V4_2, fitsv3v4):
#     rec.fit(**fit)
    
# recsToFitV3V4_3 = []
# for rec, init in zip(recsv3v4, initsV3V4_3):
#     recsToFitV3V4_3.append(rec(**init))
# for rec, fit in zip(recsToFitV3V4_3, fitsv3v4):
#     rec.fit(**fit)
    
# #passo URM_train2 e recstoFit2
ww = [0.264518,
 0.142308,
 0.502272,
 0.27785,
 0.23341,
 0.646702,
 0.94445,
 0.217856]
recommender1 = WeightedHybridScoreRecommender(URM_train, recs, inits)
recommender1.fit(fits,ww.copy())



WeightedHybridScoreRecommender: URM Detected 69 (0.87 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2415 (9.30 %) cold items.
TopPopRecommender: URM Detected 69 (0.87 %) cold users.
TopPopRecommender: URM Detected 2415 (9.30 %) cold items.
P3alphaRecommender: URM Detected 69 (0.87 %) cold users.
P3alphaRecommender: URM Detected 2415 (9.30 %) cold items.
RP3betaRecommender: URM Detected 69 (0.87 %) cold users.
RP3betaRecommender: URM Detected 2415 (9.30 %) cold items.
UserKNNCFRecommender: URM Detected 69 (0.87 %) cold users.
UserKNNCFRecommender: URM Detected 2415 (9.30 %) cold items.
ItemKNNCFRecommender: URM Detected 69 (0.87 %) cold users.
ItemKNNCFRecommender: URM Detected 2415 (9.30 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 69 (0.87 %) cold users.
IALSRecommender: URM Detected 2415 (9.30 %) cold items.
ItemKNNCBFRecommender: URM Detected 69 (0.87 %)

SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 1.85 min
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 1.87 min
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 1.90 min
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 1.91 min
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 1.92 min
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 1.94 min
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 1.95 min
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 1.96 min
SLIM_BPR_Recommender: Epoch 116 of 254. Elapsed time 1.98 min
SLIM_BPR_Recommender: Epoch 117 of 254. Elapsed time 1.99 min
SLIM_BPR_Recommender: Epoch 118 of 254. Elapsed time 2.01 min
SLIM_BPR_Recommender: Epoch 119 of 254. Elapsed time 2.03 min
SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 2.04 min
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 2.06 min
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 2.09 min
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 2.11 min
SLIM_BPR

SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 3.32 min
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 3.32 min
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 3.32 min
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 3.32 min
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 3.32 min
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 249 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 250 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 251 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 252 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 3.33 min
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 4.38 min
Deallocating Cython objects
--------FITTING IN PROGRESS: ItemK

In [18]:
recommender2 = WeightedHybridScoreRecommender(URM_train2, recs, inits2)
recommender2.fit(fits,ww.copy())

WeightedHybridScoreRecommender: URM Detected 72 (0.91 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2383 (9.17 %) cold items.
TopPopRecommender: URM Detected 72 (0.91 %) cold users.
TopPopRecommender: URM Detected 2383 (9.17 %) cold items.
P3alphaRecommender: URM Detected 72 (0.91 %) cold users.
P3alphaRecommender: URM Detected 2383 (9.17 %) cold items.
RP3betaRecommender: URM Detected 72 (0.91 %) cold users.
RP3betaRecommender: URM Detected 2383 (9.17 %) cold items.
UserKNNCFRecommender: URM Detected 72 (0.91 %) cold users.
UserKNNCFRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCFRecommender: URM Detected 72 (0.91 %) cold users.
ItemKNNCFRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 72 (0.91 %) cold users.
IALSRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCBFRecommender: URM Detected 72 (0.91 %)

SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 8.21 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 8.26 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 8.31 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 8.34 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 8.39 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 8.44 sec
SLIM_BPR_Recommender: Epoch 116 of 254. Elapsed time 8.61 sec
SLIM_BPR_Recommender: Epoch 117 of 254. Elapsed time 8.69 sec
SLIM_BPR_Recommender: Epoch 118 of 254. Elapsed time 8.75 sec
SLIM_BPR_Recommender: Epoch 119 of 254. Elapsed time 8.80 sec
SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 8.85 sec
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 8.93 sec
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 9.00 sec
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 9.06 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 9.09 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 9.14 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 14.49 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 14.63 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 14.67 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 14.71 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 14.73 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 14.76 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 14.79 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 14.84 sec
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 14.94 sec
SLIM_BPR_Recommender: Epoch 249 of 254. Elapsed time 15.03 sec
SLIM_BPR_Recommender: Epoch 250 of 254. Elapsed time 15.10 sec
SLIM_BPR_Recommender: Epoch 251 of 254. Elapsed time 15.18 sec
SLIM_BPR_Recommender: Epoch 252 of 254. Elapsed time 15.24 sec
SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 15.29 sec
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 15.35 sec
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed

In [19]:
recommender3 = WeightedHybridScoreRecommender(URM_train3, recs, inits3)
recommender3.fit(fits,ww.copy())

WeightedHybridScoreRecommender: URM Detected 64 (0.81 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2412 (9.29 %) cold items.
TopPopRecommender: URM Detected 64 (0.81 %) cold users.
TopPopRecommender: URM Detected 2412 (9.29 %) cold items.
P3alphaRecommender: URM Detected 72 (0.91 %) cold users.
P3alphaRecommender: URM Detected 2383 (9.17 %) cold items.
RP3betaRecommender: URM Detected 72 (0.91 %) cold users.
RP3betaRecommender: URM Detected 2383 (9.17 %) cold items.
UserKNNCFRecommender: URM Detected 72 (0.91 %) cold users.
UserKNNCFRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCFRecommender: URM Detected 72 (0.91 %) cold users.
ItemKNNCFRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 72 (0.91 %) cold users.
IALSRecommender: URM Detected 2383 (9.17 %) cold items.
ItemKNNCBFRecommender: URM Detected 72 (0.91 %)

SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 3.22 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 3.24 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 3.29 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 3.31 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 3.34 sec
SLIM_BPR_Recommender: Epoch 116 of 254. Elapsed time 3.36 sec
SLIM_BPR_Recommender: Epoch 117 of 254. Elapsed time 3.39 sec
SLIM_BPR_Recommender: Epoch 118 of 254. Elapsed time 3.41 sec
SLIM_BPR_Recommender: Epoch 119 of 254. Elapsed time 3.43 sec
SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 3.46 sec
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 3.48 sec
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 3.50 sec
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.53 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.55 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.58 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 6.38 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 6.41 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 6.43 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 6.45 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 6.48 sec
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 6.50 sec
SLIM_BPR_Recommender: Epoch 249 of 254. Elapsed time 6.52 sec
SLIM_BPR_Recommender: Epoch 250 of 254. Elapsed time 6.55 sec
SLIM_BPR_Recommender: Epoch 251 of 254. Elapsed time 6.57 sec
SLIM_BPR_Recommender: Epoch 252 of 254. Elapsed time 6.59 sec
SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 6.62 sec
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 6.64 sec
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 1.06 min
Deallocating Cython objects
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 1703.09 column/sec, elapsed time 0.25 min
Similarity column 25975 

In [20]:
ww = [0.264518,
 0.142308,
 0.502272,
 0.27785,
 0.13342,
 0.646702,
 0.94445,
 0.217856]

recommender1.setWeights(ww.copy())
recommender2.setWeights(ww.copy())
recommender3.setWeights(ww.copy())

In [22]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender1)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")

result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

#mapsv3v4.append(mappozzo)

EvaluatorHoldout: Processed 3000 ( 52.95% ) in 36.96 sec. Users per second: 81
EvaluatorHoldout: Processed 5666 ( 100.00% ) in 1.03 min. Users per second: 92
MAP  recommeder test 1: 0.07529601682942184

EvaluatorHoldout: Processed 4000 ( 70.80% ) in 37.33 sec. Users per second: 107
EvaluatorHoldout: Processed 5650 ( 100.00% ) in 52.93 sec. Users per second: 107
MAP  recommeder test 2: 0.07091533173577515

EvaluatorHoldout: Processed 3000 ( 53.33% ) in 30.51 sec. Users per second: 98
EvaluatorHoldout: Processed 5625 ( 100.00% ) in 1.03 min. Users per second: 91
MAP  recommeder test 2: 0.4720082156154636


AVERAGE: [0.2060731880602202]


In [23]:
ww = [0.264518,
 0.142308,
 0.502272,
 0.27785,
 0.23341,
 0.646702,
 0.94445,
 0.217856]

recommender1.setWeights(ww.copy())
recommender2.setWeights(ww.copy())
result_dict, _ = evaluator_test.evaluateRecommender(recommender1)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")

result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")
mappozzo=(MAP+MAP2)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 4000 ( 70.60% ) in 39.87 sec. Users per second: 100
EvaluatorHoldout: Processed 5666 ( 100.00% ) in 57.70 sec. Users per second: 98
MAP  recommeder test 1: 0.07520575133022181

EvaluatorHoldout: Processed 4000 ( 70.80% ) in 35.05 sec. Users per second: 114
EvaluatorHoldout: Processed 5650 ( 100.00% ) in 49.28 sec. Users per second: 115
MAP  recommeder test 2: 0.07150855420995003


AVERAGE: [0.048904768513390616]


In [24]:
0.07520575133022181 + 0.07150855420995003 - 0.07529601682942184 - 0.07091533173577515

0.0005029569749748497

In [25]:
ww = [0.20167983999999997, 0.23687631999999997,0, 0.7848215199999999, 0.30086991999999996, 
      0.6200379999999999, 0.85121488, 0.28247175999999996, 0]
recommender1.normalize_scores = False
recommender1.setWeights(ww.copy())
recommender2.normalize_scores = False
recommender2.setWeights(ww.copy())
result_dict, _ = evaluator_test.evaluateRecommender(recommender1)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")

result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")
mappozzo=(MAP+MAP2)/2
print("\nAVERAGE: ["+str(mappozzo)+"]")


EvaluatorHoldout: Processed 2000 ( 35.30% ) in 55.14 sec. Users per second: 36
EvaluatorHoldout: Processed 3000 ( 52.95% ) in 1.48 min. Users per second: 34
EvaluatorHoldout: Processed 4000 ( 70.60% ) in 2.15 min. Users per second: 31
EvaluatorHoldout: Processed 5666 ( 100.00% ) in 3.24 min. Users per second: 29
MAP  recommeder test 1: 0.0725544848958265

EvaluatorHoldout: Processed 1000 ( 17.70% ) in 39.39 sec. Users per second: 25
EvaluatorHoldout: Processed 3000 ( 53.10% ) in 1.50 min. Users per second: 33
EvaluatorHoldout: Processed 5000 ( 88.50% ) in 2.38 min. Users per second: 35
EvaluatorHoldout: Processed 5650 ( 100.00% ) in 2.60 min. Users per second: 36
MAP  recommeder test 2: 0.06994608798944027


AVERAGE: [0.07125028644263338]


In [16]:
#check num 4 config 
#validation MAP: 0.069
#test MAP: 
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.7287133469006986, 'w1': 2.4177460125941046, 'w2': 0.0, 'w3': 3.383500672092017, 'w4': 7.060286086536979, 'w5': 6.31774101047705}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.7287133469006986, 'w1': 2.4177460125941046, 'w2': 0.0, 'w3': 3.383500672092017, 'w4': 7.060286086536979, 'w5': 6.31774101047705}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.7287133469006986, 'w1': 2.4177460125941046, 'w2': 0.0, 'w3': 3.383500672092017, 'w4': 7.060286086536979, 'w5': 6.31774101047705}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [17]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsv3v4.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.09 sec. Users per second: 465
MAP  recommeder test 1: 0.07249491442136032

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.10 sec. Users per second: 466
MAP  recommeder test 2: 0.07436177597804568

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 12.18 sec. Users per second: 460
MAP  recommeder test 2: 0.07359922053810589


AVERAGE: [0.07348530364583729]


## Altro tuning di 6 recs (V3v4), si chiama old3


In [18]:
#check num 5 config Config 565:
#validation MAP 0.069
#test MAP 
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.7481869432976977, 'w1': 2.3337409830150553, 'w2': 0.0, 'w3': 2.9382858225105175, 'w4': 8.035019268731672, 'w5': 7.581387976941286}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.7481869432976977, 'w1': 2.3337409830150553, 'w2': 0.0, 'w3': 2.9382858225105175, 'w4': 8.035019268731672, 'w5': 7.581387976941286}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.7481869432976977, 'w1': 2.3337409830150553, 'w2': 0.0, 'w3': 2.9382858225105175, 'w4': 8.035019268731672, 'w5': 7.581387976941286}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [19]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsv3v4.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.00 sec. Users per second: 468
MAP  recommeder test 1: 0.07299384236066368

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 11.96 sec. Users per second: 472
MAP  recommeder test 2: 0.07393042752916315

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 11.88 sec. Users per second: 472
MAP  recommeder test 2: 0.07329653441938437


AVERAGE: [0.07340693476973707]


In [20]:
# best ever del porcodio
#check num 9config 533
#validation MAP: 0.069
#test MAP: 0.7
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.6212451948566255, 'w1': 2.748732488030105, 'w2': 2.2478782502509183, 'w3': 3.725136412423386, 'w4': 7.572634037577704, 'w5': 7.356291524110624}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.6212451948566255, 'w1': 2.748732488030105, 'w2': 2.2478782502509183, 'w3': 3.725136412423386, 'w4': 7.572634037577704, 'w5': 7.356291524110624}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.6212451948566255, 'w1': 2.748732488030105, 'w2': 2.2478782502509183, 'w3': 3.725136412423386, 'w4': 7.572634037577704, 'w5': 7.356291524110624}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [21]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsv3v4.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.20 sec. Users per second: 460
MAP  recommeder test 1: 0.07248526710905329

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.12 sec. Users per second: 465
MAP  recommeder test 2: 0.07419961241726049

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 11.94 sec. Users per second: 470
MAP  recommeder test 2: 0.07343365115570076


AVERAGE: [0.07337284356067152]


In [22]:
#nuoviii
mapsNuovi=[]
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.6221674352675048, 'w1': 2.9, 'w2': 2.4, 'w3': 3.7111183857402077, 'w4': 7.397314268688893, 'w5': 7.436779960005684}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.6221674352675048, 'w1': 2.9, 'w2': 2.4, 'w3': 3.7111183857402077, 'w4': 7.397314268688893, 'w5': 7.436779960005684}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.6221674352675048, 'w1': 2.9, 'w2': 2.4, 'w3': 3.7111183857402077, 'w4': 7.397314268688893, 'w5': 7.436779960005684}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [23]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsNuovi.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.00 sec. Users per second: 468
MAP  recommeder test 1: 0.07281496373160914

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.13 sec. Users per second: 465
MAP  recommeder test 2: 0.07427706288229143

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 11.97 sec. Users per second: 469
MAP  recommeder test 2: 0.07361805342469388


AVERAGE: [0.07357002667953148]


In [24]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.5975452463999718, 'w1': 2.882068453840565, 'w2': 2.3057027580015435, 'w3': 3.623351583667574, 'w4': 7.3, 'w5': 7.5}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.5975452463999718, 'w1': 2.882068453840565, 'w2': 2.3057027580015435, 'w3': 3.623351583667574, 'w4': 7.3, 'w5': 7.5}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.5975452463999718, 'w1': 2.882068453840565, 'w2': 2.3057027580015435, 'w3': 3.623351583667574, 'w4': 7.3, 'w5': 7.5}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [25]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsNuovi.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 11.89 sec. Users per second: 472
MAP  recommeder test 1: 0.0727753023936397

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.04 sec. Users per second: 468
MAP  recommeder test 2: 0.07423563004156618

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 12.10 sec. Users per second: 463
MAP  recommeder test 2: 0.07363229768740362


AVERAGE: [0.07354774337420317]


In [26]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.6012505452092192, 'w1': 2.8761969880753866, 'w2': 2.2600148673878415, 'w3': 3.612973067563837, 'w4': 7.3, 'w5': 7.5}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.6012505452092192, 'w1': 2.8761969880753866, 'w2': 2.2600148673878415, 'w3': 3.612973067563837, 'w4': 7.3, 'w5': 7.5}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.6012505452092192, 'w1': 2.8761969880753866, 'w2': 2.2600148673878415, 'w3': 3.612973067563837, 'w4': 7.3, 'w5': 7.5}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [27]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsNuovi.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.01 sec. Users per second: 468
MAP  recommeder test 1: 0.07277118572767977

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 11.97 sec. Users per second: 471
MAP  recommeder test 2: 0.07422024709993025

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 11.79 sec. Users per second: 476
MAP  recommeder test 2: 0.07363760378559614


AVERAGE: [0.07354301220440206]


In [28]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.6217404317111441, 'w1': 2.887918012840853, 'w2': 2.0, 'w3': 3.635098512149486, 'w4': 7.3, 'w5': 7.417367659076582}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.6217404317111441, 'w1': 2.887918012840853, 'w2': 2.0, 'w3': 3.635098512149486, 'w4': 7.3, 'w5': 7.417367659076582}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.6217404317111441, 'w1': 2.887918012840853, 'w2': 2.0, 'w3': 3.635098512149486, 'w4': 7.3, 'w5': 7.417367659076582}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [29]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsNuovi.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.04 sec. Users per second: 466
MAP  recommeder test 1: 0.07281736227355952

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.08 sec. Users per second: 467
MAP  recommeder test 2: 0.07430388273285196

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 11.78 sec. Users per second: 476
MAP  recommeder test 2: 0.07356146953046387


AVERAGE: [0.07356090484562512]


In [30]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFitV3V4_1)
best_parameters={'w0': 1.607668536002117, 'w1': 2.8996202310531825, 'w2': 2.2597622819614824, 'w3': 3.6949045192245262, 'w4': 7.3, 'w5': 7.399871554980856}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFitV3V4_2)
best_parameters={'w0': 1.607668536002117, 'w1': 2.8996202310531825, 'w2': 2.2597622819614824, 'w3': 3.6949045192245262, 'w4': 7.3, 'w5': 7.399871554980856}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFitV3V4_3)
best_parameters={'w0': 1.607668536002117, 'w1': 2.8996202310531825, 'w2': 2.2597622819614824, 'w3': 3.6949045192245262, 'w4': 7.3, 'w5': 7.399871554980856}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 71 (0.89 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2389 (9.20 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2415 (9.30 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 72 (0.91 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2398 (9.23 %) cold items.
--------FITTING...-------


In [31]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")
mapsNuovi.append(mappozzo)

EvaluatorHoldout: Processed 5615 ( 100.00% ) in 12.28 sec. Users per second: 457
MAP  recommeder test 1: 0.07277594080077855

EvaluatorHoldout: Processed 5638 ( 100.00% ) in 12.29 sec. Users per second: 459
MAP  recommeder test 2: 0.07427026964629459

EvaluatorHoldout: Processed 5609 ( 100.00% ) in 12.18 sec. Users per second: 461
MAP  recommeder test 2: 0.07362825376679064


AVERAGE: [0.07355815473795459]


## risultati finali:

In [32]:
#new finali
# i primi 3 sono del nuov tuning
#ultimi 5 del nuovo tuning che lavora sul migliore in teoria
print("risultati v3v4:")
print(mapsv3v4)
print("risultati nuovi:")
print(mapsNuovi)

risultati v3v4:
[0.0728112860660532, 0.07280664151785322, 0.0728177202779127, 0.07348530364583729, 0.07340693476973707, 0.07337284356067152]
risultati nuovi:
[0.07357002667953148, 0.07354774337420317, 0.07354301220440206, 0.07356090484562512, 0.07355815473795459]


## rifitto tutto ecc x kaggle

In [4]:
#devo rifittare con nuovo train/test split

userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'


only_URM = {
        "URM_train" : URM_all
    }

cython = {
        "URM_train" : URM_all,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_all,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_all,
        "ICM_train" : ICM_all
    }


# 6 recomm
'''
fits = [alpha,beta,cy,iicm,ials,itemcbf]

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]
inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]
'''
# 9 recomm

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    UserKNNCFRecommender,
    UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]

inits= [
        only_URM,only_URM,only_URM,also_ICM,cython,only_URM,also_ICM,only_URM,CBF 
    ]
fits = [alpha,beta,userknn,uicm,cy,itemknn,iicm,ials,itemcbf]

In [5]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 14 (0.07 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 7947 ( 100 % ), 16999.35 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 18732.33 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2154.57 column/sec, elapsed time 0.06 min
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.02 sec


SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 3.01 sec
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 3.03 sec
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 3.06 sec
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.08 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.10 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.13 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 3.15 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 3.18 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.21 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.24 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.30 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.32 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.36 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.39 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.42 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 6.32 sec
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 6.34 sec
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 32.28 sec
Deallocating Cython objects
Similarity column 25975 ( 100 % ), 7200.51 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 7623.25 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 3921.19 column/sec, elapsed time 0.11 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.44 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 8.69 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 12.89 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 17.19 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 21.35 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 25.59 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 29.79 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 34.08 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 38.25 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 42.65 s

In [6]:
recommender=WeightedHybridV3forBayesianSearch(URM_all,ICM_all,recsToFit)
#best_parameters={'w0': 0.08440125742215467, 'w1': 0.18784463320807385, 'w2': 0.0, 'w3': 0.037743047070985955, 'w4': 0.26228260366867695, 'w5': 0.0, 'w6': 0.2274472210856471, 'w7': 1.0, 'w8': 0.6186222486444528}
#er mejor prima:
#best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
#best_parameters={'w0': 1.6212451948566255, 'w1': 2.748732488030105, 'w2': 2.2478782502509183, 'w3': 3.725136412423386, 'w4': 7.572634037577704, 'w5': 7.356291524110624}
best_parameters={'w0': 0.30662760605578626, 'w1': 0.11493515568333727, 'w2': 0.0, 'w3': 0.014240026483692825, 'w4': 0.0, 'w5': 0.026373334285914603, 'w6': 0.3013095880679471, 'w7': 0.9349266391691279, 'w8': 0.7847822337442577}

recommender.fit(**best_parameters)

WeightedHybridV3forBayesianSearch: URM Detected 1079 (4.15 %) cold items.
--------FITTING...-------


In [7]:
import time
URM_file = open(r"..\data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('OUTPUTS\outputIbridoSuperTuning9algos.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Ha finito in questo tempo: 79.07553005218506 secondi


## Versione a 6 recommenders x kaggle

In [33]:
#devo rifittare con nuovo train/test split

userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'


only_URM = {
        "URM_train" : URM_all
    }

cython = {
        "URM_train" : URM_all,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_all,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_all,
        "ICM_train" : ICM_all
    }



fits = [alpha,beta,cy,iicm,ials,itemcbf]

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]
inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]


In [34]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.01 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.05 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.08 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.12 sec
SLIM_BPR_Recommender: Epoch 5 of 254. Elapsed time 0.14 sec
SLIM_BPR_Recommender: Epoch 6 of 254. Elapsed time 0.17 sec
SLIM_BPR_Recommender: Epoch 7 of 254. Elapsed time 0.20 sec
SLIM_BPR_Recommender: Epoch 8 of 254. Elapsed time 0.23 sec
SLIM_BPR_Recommender: Epoch 9 of 254. Elapsed time 0

SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.40 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.43 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.46 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.48 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.51 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.53 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.56 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.58 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.61 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.63 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.66 sec
SLIM_BPR_Recommender: Epoch 139 of 254. Elapsed time 3.69 sec
SLIM_BPR_Recommender: Epoch 140 of 254. Elapsed time 3.71 sec
SLIM_BPR_Recommender: Epoch 141 of 254. Elapsed time 3.74 sec
SLIM_BPR_Recommender: Epoch 142 of 254. Elapsed time 3.77 sec
SLIM_BPR_Recommender: Epoch 143 of 254. Elapsed time 3.80 sec
SLIM_BPR

IALSRecommender: Epoch 3 of 45. Elapsed time 12.34 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 16.41 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 20.55 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 24.70 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 28.93 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 33.12 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 37.50 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 41.75 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 46.04 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 50.20 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 54.67 sec
IALSRecommender: Epoch 14 of 45. Elapsed time 58.85 sec
IALSRecommender: Epoch 15 of 45. Elapsed time 1.05 min
IALSRecommender: Epoch 16 of 45. Elapsed time 1.12 min
IALSRecommender: Epoch 17 of 45. Elapsed time 1.20 min
IALSRecommender: Epoch 18 of 45. Elapsed time 1.27 min
IALSRecommender: Epoch 19 of 45. Elapsed time 1.34 min
IALSRecommender: Epoch 20 of 45. Elapsed time 1.41 min
IALSR

In [35]:

from myRecommenders.WeightedHybridV4forKaggle import WeightedHybridV4forKaggle
recommender=WeightedHybridV4forKaggle(URM_all,recsToFit)
#best_parameters={'w0': 0.08440125742215467, 'w1': 0.18784463320807385, 'w2': 0.0, 'w3': 0.037743047070985955, 'w4': 0.26228260366867695, 'w5': 0.0, 'w6': 0.2274472210856471, 'w7': 1.0, 'w8': 0.6186222486444528}
#er mejor prima:
#best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
#best_parameters={'w0': 1.6212451948566255, 'w1': 2.748732488030105, 'w2': 2.2478782502509183, 'w3': 3.725136412423386, 'w4': 7.572634037577704, 'w5': 7.356291524110624}
#best_parameters={'w0': 1.7481869432976977, 'w1': 2.3337409830150553, 'w2': 0.0, 'w3': 2.9382858225105175, 'w4': 8.035019268731672, 'w5': 7.581387976941286}
best_parameters={'w0': 1.6221674352675048, 'w1': 2.9, 'w2': 2.4, 'w3': 3.7111183857402077, 'w4': 7.397314268688893, 'w5': 7.436779960005684}

recommender.fit(**best_parameters)

WeightedHybridV4forKaggle: URM Detected 1079 (4.15 %) cold items.
TopPopRecommender: URM Detected 1079 (4.15 %) cold items.
--------FITTING...-------


In [36]:
import time
URM_file = open(r"..\data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('OUTPUTS\outputIbridoSuperTuning9algosTopPop.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Ha finito in questo tempo: 28.97739815711975 secondi
